In [44]:
import json
from matplotlib.font_manager import json_dump
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import seaborn as sns
import matplotlib.pylab as plt

sf_data = []
vicon_data = []

for line in open("sensor_floor_data.txt", "r"):
    sf_data.append(json.loads(line))

df_sf = pd.DataFrame(sf_data)
df_sf.to_csv('out_1.csv')
df_sf

,timestamp,strip_id,node_id,data,column_num,ID
0,1.658244e+09,5,4,"[{'r': [0], 'm': [33.15, 54.3, -118.1999999999...",5,1
1,1.658244e+09,11,4,"[{'r': [0], 'm': [0.3, -98.85, -200.25]}, {'r'...",11,1
2,1.658244e+09,9,11,"[{'r': [0], 'm': [255.45, 52.8, -152.25]}, {'r...",9,1
3,1.658244e+09,8,3,"[{'r': [0], 'm': [19.65, -51.6, 5.7]}, {'r': [...",8,1
4,1.658244e+09,10,4,"[{'r': [0], 'm': [-251.25, -45.6, -181.2]}, {'...",10,1
...,...,...,...,...,...,...
3768,1.658244e+09,5,2,"[{'r': [0], 'm': [-50.55, 102.89999999999999, ...",5,11
3769,1.658244e+09,8,13,"[{'r': [0], 'm': [-1.65, 49.199999999999996, 0...",8,11
3770,1.658244e+09,21,1,"[{'r': [0], 'm': [65.55, 27.75, 28.65]}, {'r':...",21,11
3771,1.658244e+09,15,15,"[{'r': [0], 'm': [32.55, -38.55, 30.9]}, {'r':...",15,11


In [45]:
#1st option
df_sf_buf = df_sf.copy()
df_sf_buf = df_sf_buf.reindex(df_sf_buf.columns.tolist() + ['rssi','mx','my','mz'], axis=1) 

for index, row in df_sf_buf.T.items():
    rssi = []
    magneto = []
    for i in range(len(df_sf_buf['data'][index])):
        #print((test_df['data'][index][i]['r']))
        rssi.append(df_sf_buf['data'][index][i]['r'])
        magneto.append(df_sf_buf['data'][index][i]['m'])
    
    rssi_avg = np.mean(rssi, axis=0)
    magneto_avg = np.mean(magneto, axis=0)
    df_sf_buf['rssi'][index] = rssi_avg[0]
    df_sf_buf['mx'][index] = magneto_avg[0]
    df_sf_buf['my'][index] = magneto_avg[1]
    df_sf_buf['mz'][index] = magneto_avg[2]
    rssi.clear()
    magneto.clear()
    #print(rssi_avg, magneto_avg)

#test_df['data'][0][0]['m']
df_sf_buf = df_sf_buf.drop(columns=['data'])
df_sf_buf.to_csv('out_2.csv')
df_sf_buf

/tmp/ipykernel_6791/3793073884.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sf_buf['rssi'][index] = rssi_avg[0]
/tmp/ipykernel_6791/3793073884.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sf_buf['mx'][index] = magneto_avg[0]
/tmp/ipykernel_6791/3793073884.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sf_buf['my'][index] = magneto_avg[1]
/tmp/ipykernel_6791/3793073884.py:18: SettingWithCopyWarning: 
A val

,timestamp,strip_id,node_id,column_num,ID,rssi,mx,my,mz
0,1.658244e+09,5,4,5,1,0.0,34.231579,54.181579,-117.647368
1,1.658244e+09,11,4,11,1,0.0,0.221053,-99.726316,-201.434211
2,1.658244e+09,9,11,9,1,0.0,255.576316,52.886842,-152.700000
3,1.658244e+09,8,3,8,1,0.0,18.742105,-50.850000,5.415789
4,1.658244e+09,10,4,10,1,0.0,-250.721053,-45.426316,-180.410526
...,...,...,...,...,...,...,...,...,...
3768,1.658244e+09,5,2,5,11,0.0,-50.581579,102.094737,-74.194737
3769,1.658244e+09,8,13,8,11,0.0,-2.928947,49.271053,2.052632
3770,1.658244e+09,21,1,21,11,0.0,65.875000,28.475000,29.625000
3771,1.658244e+09,15,15,15,11,0.0,32.021053,-37.547368,32.052632


In [54]:
#2nd option but slower
df_sf_buf = df_sf.explode('data')
df_sf_proc = pd.json_normalize(json.loads(df_sf_buf.to_json(orient="records")))
df_sf_proc.rename(columns={'data.r':'rssi','data.m':'magnetometer'}, inplace=True)

df_sf_proc[['mx','my','mz']] = pd.DataFrame(df_sf_proc.magnetometer.to_list(), index = df_sf_proc.index)
df_sf_proc['rssi'] = df_sf_proc['rssi'].str.get(0)
df_sf_proc = df_sf_proc.drop(columns=['magnetometer'])
df_sf_proc[['strip_id', 'node_id']] = df_sf_proc[['strip_id', 'node_id']].astype(int)
df_sf_proc.to_csv('out_2.csv')
df_sf_proc

,timestamp,strip_id,node_id,column_num,ID,rssi,mx,my,mz
0,1.658244e+09,5,4,5,1,0,33.15,54.30,-118.20
1,1.658244e+09,5,4,5,1,0,34.65,53.70,-118.65
2,1.658244e+09,5,4,5,1,0,34.20,53.55,-117.45
3,1.658244e+09,5,4,5,1,0,34.35,54.00,-117.60
4,1.658244e+09,5,4,5,1,0,34.20,53.25,-116.40
...,...,...,...,...,...,...,...,...,...
71030,1.658244e+09,12,11,12,11,0,17.25,91.05,75.00
71031,1.658244e+09,12,11,12,11,0,18.30,90.45,74.25
71032,1.658244e+09,12,11,12,11,0,18.15,90.90,74.85
71033,1.658244e+09,12,11,12,11,0,18.00,90.45,74.25


In [57]:
df_sf_final = df_sf_proc.copy()
#df_sf_final = df_sf_final.groupby(df_sf_final.index // 19).mean()
df_sf_final = df_sf_final.groupby(df_sf_final.index // 19)['rssi','mx','my','mz'].mean()
#df_sf_final[['strip_id','node_id','column_num','ID']] = df_sf_final[['strip_id','node_id','column_num','ID']].round(0)
df_sf_final.to_csv('out_3_test.csv')
df_sf_final

/tmp/ipykernel_4707/1701679357.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_sf_final = df_sf_final.groupby(df_sf_final.index // 19)['rssi','mx','my','mz'].mean()


,rssi,mx,my,mz
0,0.0,34.231579,54.181579,-117.647368
1,0.0,0.221053,-99.726316,-201.434211
2,0.0,255.576316,52.886842,-152.700000
3,0.0,18.742105,-50.850000,5.415789
4,0.0,-250.721053,-45.426316,-180.410526
...,...,...,...,...
3734,0.0,-38.060526,88.113158,-54.244737
3735,0.0,15.157895,43.657895,9.347368
3736,0.0,55.144737,7.760526,30.284211
3737,0.0,27.615789,3.063158,45.813158


In [ ]:
# #df_test = test2.groupby(test2.index // 19).mean()
# #df_test[f'r_avg'] = df_test.shift(-1).rolling(4,4).mean()
# df_test = test2.copy()
# #df_test = pd.DataFrame(np.einsum('ijk->ik',df_test.values.reshape(-1,9,df_test.shape[1]))/19.0)
# df_test = df_test.groupby(df_test.index // 19).mean()
# df_test[['strip_id','node_id','column_num','ID']] = df_test[['strip_id','node_id','column_num','ID']].round(0)
# #df_test['timestamp'] = pd.to_datetime(df_test['timestamp'],unit='s')
# df_test.to_csv('out2.csv')
# df_test

In [35]:
for line in open("vicon_data.txt", "r"):
    vicon_data.append(json.loads(line))

df_vicon = pd.DataFrame(vicon_data)
df_vicon['time'] = pd.to_datetime(df_vicon['time'],unit='s')
df_vicon

,translation,object,rotation,time
0,"[-0.019098650364350662, -3.530711067039381, 0....",/vicon/aerobie/aerobie,"[0.002791165396519843, 0.021542131367492525, 0...",2019-12-20 15:44:13.228681984
1,"[-0.019094903550087375, -3.530702772720012, 0....",/vicon/aerobie/aerobie,"[0.0031022221734463536, 0.021847041154000015, ...",2019-12-20 15:44:13.235233024
2,"[-0.019107108550540985, -3.5307083485220696, 0...",/vicon/aerobie/aerobie,"[0.0028209960975453994, 0.021537958612233326, ...",2019-12-20 15:44:13.240809984
3,"[-0.019153586479018256, -3.530696099263355, 0....",/vicon/aerobie/aerobie,"[0.002787947730297147, 0.021247795200080816, 0...",2019-12-20 15:44:13.243851776
4,"[-0.019111224132754453, -3.530693922743832, 0....",/vicon/aerobie/aerobie,"[0.003156992729313418, 0.021101027269709416, 0...",2019-12-20 15:44:13.246320896
...,...,...,...,...
40613,"[-4.857236229945065, -6.046858107129603, 1.970...",/vicon/aerobie/aerobie,"[-0.019792360484122168, 0.06284711223687622, 0...",2019-12-20 15:46:09.582201856
40614,"[-4.853490351279589, -6.052476897197011, 1.969...",/vicon/aerobie/aerobie,"[-0.022654007259853923, 0.07020377600416657, 0...",2019-12-20 15:46:09.587290112
40615,"[-4.848750056582673, -6.0569039767523405, 1.96...",/vicon/aerobie/aerobie,"[-0.01707798853506186, 0.07151641977704917, 0....",2019-12-20 15:46:09.592329984
40616,"[-4.846571202942883, -6.0623760352573335, 1.96...",/vicon/aerobie/aerobie,"[-0.0098840680176943, 0.09196323504237189, 0.4...",2019-12-20 15:46:09.596987136
